In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py
import os
from photoz_utils import *
GB_LIMIT = 2

In [2]:
gpus = tf.config.list_physical_devices('GPU')

if gpus:
  # Restrict TensorFlow to only allocate 2GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1000*GB_LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
class Categorization:
    def __init__(
        self,
        index,
        object_id,
        num_galaxies = 0,
        merging = False,
        multiple_galaxies = False,
        multiple_galaxies_all_bands = False,
        smooth_central_galaxy = False,
        defined_central_galaxy = False,
        off_center_central_galaxy = False,
        faint_central_galaxy = False,
        nearby_bright = False,
        grainy_image = False,
        edge_on = False
    ): 
        self.index: int = index
        self.object_id: int = object_id
        self.num_galaxies: int = num_galaxies
        self.merging: bool = merging
        self.multiple_galaxies: bool = multiple_galaxies
        self.multiple_galaxies_all_bands: bool = multiple_galaxies_all_bands
        self.smooth_central_galaxy: bool = smooth_central_galaxy
        self.defined_central_galaxy: bool = defined_central_galaxy
        self.off_center_central_galaxy: bool = off_center_central_galaxy
        self.faint_central_galaxy: bool = faint_central_galaxy
        self.nearby_bright: bool = nearby_bright
        self.grainy_image: bool = grainy_image
        self.edge_on: bool = edge_on
    
    def __compose_dataframe__(self):
        d = {
            'index': [self.index],
            'object_id': [self.object_id],
            'num_galaxies': [self.num_galaxies],
            'merging': [self.merging],
            'multiple_galaxies': [self.multiple_galaxies],
            'multiple_galaxies_all_bands': [self.multiple_galaxies_all_bands],
            'smooth_central_galaxy': [self.smooth_central_galaxy],
            'defined_central_galaxy': [self.defined_central_galaxy],
            'off_center_central_galaxy': [self.off_center_central_galaxy],
            'faint_central_galaxy': [self.faint_central_galaxy],
            'nearby_bright': [self.nearby_bright],
            'grainy_image': [self.grainy_image],
            'edge_on': [self.edge_on]
        }
        return pd.DataFrame(data=d)

In [4]:
df = pd.read_csv('/data/HSC/HSC_v6/CNN_visualization_v1/selected_imgs_data.csv')

In [5]:
df

,Unnamed: 0,object_id,specz,photoz_nn,bias_nn,g_cmodel_mag,g_cmodel_magsigma,r_cmodel_mag,r_cmodel_magsigma,i_cmodel_mag,...,z_cmodel_mag,z_cmodel_magsigma,y_cmodel_mag,y_cmodel_magsigma,photoz_rf,bias_rf,photoz_cnn,bias_cnn,cnn_nn_diff,cnn_rf_diff
0,40074,70365041915877073,0.209333,2.298409,1.727462,20.939396,0.006759,20.391289,0.004909,20.199217,...,20.173031,0.005642,20.224075,0.009329,1.571517,1.126393,0.205743,0.002968,-1.724494,-1.123425
1,16490,41579711536596015,0.094230,1.803216,1.561816,19.777716,0.001744,19.451578,0.001427,19.375889,...,19.069920,0.002298,18.850060,0.004101,1.292686,1.095251,0.094078,0.000139,-1.561677,-1.095111
2,10864,41021550471707938,0.065730,1.515024,1.359908,20.976364,0.004788,20.924738,0.005857,20.642412,...,20.728413,0.009812,20.506249,0.018614,1.733122,1.564554,0.066774,0.000980,-1.358928,-1.563574
3,6090,39935241573388804,0.068960,1.359743,1.207513,19.923796,0.004506,19.669414,0.005188,19.481480,...,19.345255,0.037464,19.463173,0.060340,1.584030,1.417330,0.068808,0.000142,-1.207371,-1.417189
4,5432,39600448872667363,0.136283,1.502387,1.202257,20.204815,0.003969,20.016647,0.003717,19.693857,...,19.837912,0.006127,19.896181,0.009943,1.527940,1.224746,0.133454,0.002489,-1.199768,-1.222256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,13719,41210675061614649,0.161940,0.158991,0.002538,19.813929,0.001492,19.011013,0.000910,18.552931,...,18.268175,0.000816,18.050549,0.001258,0.165070,0.002694,0.110797,0.044015,0.041477,0.041321
69,7391,40146085812923356,0.299930,0.296162,0.002899,20.562515,0.002694,19.188488,0.001305,18.544563,...,18.177872,0.001144,17.925365,0.001930,0.297384,0.001958,0.244237,0.042843,0.039944,0.040885
70,12656,41139473093789210,0.470460,0.469645,0.000555,22.373089,0.007589,20.648031,0.003145,19.806511,...,19.448818,0.001547,19.267090,0.002385,0.466960,0.002380,0.529261,0.039988,0.039434,0.037609
71,16135,41570778004587027,0.635600,0.635401,0.000121,23.770222,0.014516,22.977875,0.015730,22.408157,...,22.197567,0.014902,22.030788,0.020723,0.634176,0.000871,0.573059,0.038237,0.038116,0.037367


In [6]:
hf = h5py.File('/data/HSC/HSC_v6/step2A/127x127/five_band_image127x127_with_metadata_corrected_testing.hdf5', 'r')

In [7]:
# match indices of object id's
imgs_idx = df['Unnamed: 0']
len(imgs_idx)

73

In [432]:
gal = 73
current_idx = df['Unnamed: 0'][gal]
current_oid = df.object_id[gal]

fig, ax = plt.subplots(5, figsize=(30,30))
fig.suptitle(f'Object ID: {current_oid}, Index: {current_idx}', y=0.9)

for i in range(0, 5):
    ax[i].imshow(hf['image'][imgs_idx[gal], i, :, :])
    

KeyError: 73

In [427]:
cat_args = {
    'num_galaxies': 1,
    'merging': False,
    'multiple_galaxies': False,
    'multiple_galaxies_all_bands': False,
    'smooth_central_galaxy': False,
    'defined_central_galaxy': True,
    'off_center_central_galaxy': False,
    'faint_central_galaxy': False,
    'nearby_bright': False,
    'grainy_image': False,
    'edge_on': False
}
cat = Categorization(current_idx, current_oid, **cat_args).__compose_dataframe__()

In [428]:
cat

,index,object_id,num_galaxies,merging,multiple_galaxies,multiple_galaxies_all_bands,smooth_central_galaxy,defined_central_galaxy,off_center_central_galaxy,faint_central_galaxy,nearby_bright,grainy_image,edge_on
0,29833,42692670706968750,1,False,False,False,False,True,False,False,False,False,False


In [429]:
cat_df = cat_df.append(cat)

In [430]:
cat_df

,index,object_id,num_galaxies,merging,multiple_galaxies,multiple_galaxies_all_bands,smooth_central_galaxy,defined_central_galaxy,off_center_central_galaxy,faint_central_galaxy,nearby_bright,grainy_image,edge_on
0,40074,70365041915877073,2,False,True,True,False,True,False,False,True,False,False
0,16490,41579711536596015,2,False,True,True,False,True,False,False,True,False,True
0,10864,41021550471707938,8,True,True,True,False,True,False,False,True,False,True
0,6090,39935241573388804,3,False,True,True,False,True,False,False,True,True,False
0,5432,39600448872667363,2,False,True,True,False,True,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,13719,41210675061614649,3,False,True,True,False,True,False,False,False,False,False
0,7391,40146085812923356,3,False,True,True,False,True,False,False,True,False,False
0,12656,41139473093789210,3,False,True,True,False,True,False,False,True,False,False
0,16135,41570778004587027,6,False,True,True,True,False,False,False,True,False,False


In [431]:
cat_df.to_csv('/data/HSC/HSC_v6/CNN_visualization_v1/selected_imgs_cat.csv', index=False)

In [13]:
# default
# cat_args = {
#     'num_galaxies': 0,
#     'merging': False,
#     'multiple_galaxies': False,
#     'multiple_galaxies_all_bands': False,
#     'smooth_central_galaxy': False,
#     'defined_central_galaxy': False,
#     'off_center_central_galaxy': False,
#     'faint_central_galaxy': False,
#     'nearby_bright': False,
#     'grainy_image': False,
#     'edge_on': False
# }